In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
#交叉熵在调整权值或偏度值时的策略比较合理，训练模型收敛的速度会比较快
# 载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True) #把MNIST_data存在文件当前目录下，否者写明绝对路径

#每个批次的大小，训练模型时以批次为单位放入图片
batch_size = 100 #每次训练放入100张图片
#计算一共多少批次
n_batch = mnist.train.num_examples // batch_size  # // 代表整除

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784]) #None在执行时是100，784是因为每张图片被转换成1 * 784的向量
y = tf.placeholder(tf.float32,[None,10]) #y是标签，数字是0-9
keep_prob = tf.placeholder(tf.float32)

#创建一个简单的神经网络
W1 = tf.Variable(tf.truncated_normal([784,2000],stddev=0.1)) #每张图片有784个像素点，输入到神经网络中是一张一张输入的
b1 = tf.Variable(tf.zeros([2000]) + 0.1)
L1 = tf.nn.tanh(tf.matmul(x,W1) + b1)
L1_drop = tf.nn.dropout(L1,keep_prob) #keep_prob代表的是有百分之几的神经元工作

W2 = tf.Variable(tf.truncated_normal([2000,2000],stddev=0.1)) 
b2 = tf.Variable(tf.zeros([2000]) + 0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2) + b2)
L2_drop = tf.nn.dropout(L2,keep_prob) #keep_prob代表的是有百分之几的神经元工作

W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1)) 
b3 = tf.Variable(tf.zeros([1000]) + 0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop,W3) + b3)
L3_drop = tf.nn.dropout(L3,keep_prob) #keep_prob代表的是有百分之几的神经元工作

W4 = tf.Variable(tf.truncated_normal([1000,10],stddev=0.1)) 
b4 = tf.Variable(tf.zeros([10]) + 0.1)
Wx_plus_b_L = tf.matmul(L3_drop,W4) + b4

prediction = tf.nn.softmax(tf.matmul(L3_drop,W4) + b4)

#交叉熵代价函数
#loss = tf.reduce_mean(tf.square(y - prediction))    ctrl+?可以注释
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=Wx_plus_b_L))
#使用梯度下降
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) #比较两个参数是否相等，返回boolean型，argmax可以返回最大的数值在第几个位置，1表示按行查找
#argmax返回一维张量中最大的值所在的位置 https://blog.csdn.net/u013580539/article/details/79339250
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #把boolean型转换成float型，并求平均值

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):   #把所有的图片训练21次
         for batch in range(n_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size) #图片的数据保存在batch_xs, 图片的标签保存在batch_ys，自动获取下一批次
                sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
                
         test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
         train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
         print("Iter" + str(epoch) + "Testing Accuracy" + str(test_acc) + "Training Accurary" + str(train_acc))